In [ ]:
from collections import defaultdict
from os import listdir
from os.path import isfile, join
from os import path
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy.ndimage.interpolation import rotate
from keras.utils import np_utils
#import random

In [ ]:
RESULTS_PATH = "RESULTS/"
DATASET_PATH = "DATASET/PUBLIC/"
SOLVERS = [
    #"GREEDY",
    "CLK",
    "LKH",
    "CONCORDE",
    "EAX",
    "MAOS",
]
SOLVERS_NAME = [
    "Chained Lin-Kernighan",
    "Lin-Kernighan-Helsgaun",
    "Concorde",
    "Edge Assembly Crossover",
    "Multiagent optimization system"
]
DATASETS = {
    "TSPLIB",
    "TNM",
    "NATIONAL",
    "VLSI",
    "NETGEN",
    "NETGENM",
    "RUE",
    "ART",
    "TSPGEN"
}
MAX_INT = 9223372036854775807

In [ ]:
def create_timesteps(n_seconds, n_timesteps):
    timesteps = np.logspace(1, 20, num=n_timesteps+1, base=2)-2 # escala logaritmica para los tiempos
    timesteps = n_seconds*timesteps/timesteps.max()
    timesteps = timesteps[1:]
    return timesteps

In [ ]:
timesteps = create_timesteps(7200, 200)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [ ]:
y_train = np.load('/content/drive/My Drive/Colab Notebooks/TSP algorithm selection/y_train.npy')

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

## Preparing Frames

In [ ]:
set_limit = 5

In [ ]:
new_X_train = []

for frame in X_train:
    new_frame = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            if frame[i,j] > set_limit:
                new_frame[i,j] = 1
            else:
                new_frame[i,j] = frame[i,j]/5
    new_X_train.append(new_frame)

In [ ]:
np.array(new_X_train).shape

In [ ]:
np.array(new_X_train).max()

## Preparing labels

In [ ]:
def get_single_ranking(timeline,time_):
    best_ = []
    for num in range(5):
        best_.append([*timeline[num][time_],num])
    if [i[0] for i in best_] == [MAX_INT]*5:
        return -1
    #print([truncate(i, 0) for i in np.array(best_)[:,0]])
    best_.sort(key=lambda x: x[1])
    best_.sort(key=lambda x: x[0])
    #print([truncate(i, 0) for i in np.array(best_)[:,0]])
    return list(np.array(best_)[:,2])
    #return list(np.array([best_ov[s] for s in SOLVERS]).argsort())

def get_y_rank(y, categorical=False):
    new_y = []
    for y_ in y:
        timeline = []
        for ts in range(len(timesteps)):
            rank = get_single_ranking(y_,ts)
            if rank == -1:
                if categorical:
                    timeline.append(np_utils.to_categorical(0, 6))
                else:
                    timeline.append(0)
            else:
                if categorical:
                    timeline.append(np_utils.to_categorical(rank[0]+1, 6))
                else:
                    timeline.append(rank[0]+1)
        new_y.append(timeline)
    new_y = np.array(new_y)
    return new_y

In [ ]:
cat_y_train = get_y_rank(y_train, categorical=True)

In [ ]:
cat_y_train.shape

## Neural Network

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Reshape, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras import activations

In [ ]:
new_X_train=np.array(new_X_train)

In [ ]:
# Flatten
N_grid = 100
new_X_train = new_X_train.reshape(len(new_X_train), N_grid**2)

### Fully Connected

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(N_grid**2,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
#model.add(Reshape((200,6)))
model.add(Dense(200))
#model.add(Activation('softmax'))
model.add(Activation('linear'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')#categorical_crossentropy
history = model.fit(new_X_train, cat_y_train, batch_size=256, epochs=250, verbose=2, validation_split = 0.3)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### RNN

In [ ]:
from keras.layers import Bidirectional, LSTM, RepeatVector
from keras.models import Sequential, Model
from keras.layers import Reshape
from keras.layers import Input

In [ ]:
model = Sequential()
#model.add(LSTM(64, input_shape=(1, 10000)))
model.add(Dense(512, input_shape=(10000,)))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
#model.add(LSTM(64))
model.add(Dense(6))
model.add(Activation('softmax'))
model.summary()

In [ ]:
binary_y = np_utils.to_categorical(cat_y_train[:,-1], 6)

In [ ]:
summy = [0,0,0,0,0,0]
for i in binary_y:
  summy+=i

In [ ]:
summy

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
#model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
history = model.fit(new_X_train.reshape(len(new_X_train),10000), binary_y, batch_size=512, epochs=50, verbose=2, validation_split = 0.3)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:

inputs = Input(shape=(10000,))

model = Dense(512)(inputs)
model = RepeatVector(256)(model)
model = LSTM(256, return_sequences=True)(model)
#model = LSTM(64, return_sequences=True)(model)
#model = RepeatVector(200)(model)
#model = RepeatVector(200)(model)
model = LSTM(200, return_sequences=False)(model)

#model = RepeatVector(6)(model)

model = Model(inputs, model)
model.summary()

In [ ]:
model.compile(loss='mse',optimizer='adam', metrics=['acc'])

In [ ]:
new_X_train.shape

In [ ]:
cat_y_train.min()

In [ ]:
np.save("new_X_train.npy", new_X_train.reshape(len(new_X_train),10000,1))
np.save("cat_y_train.npy", cat_y_train)

In [ ]:
new_X_train = np.load("/content/drive/My Drive/Colab Notebooks/TSP algorithm selection/new_X_train.npy")
cat_y_train = np.load("/content/drive/My Drive/Colab Notebooks/TSP algorithm selection/cat_y_train.npy")

In [ ]:
history = model.fit(new_X_train.reshape(len(new_X_train),10000), cat_y_train, batch_size=254, epochs=20, verbose=2, validation_split = 0.3)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Predict

In [ ]:
X_test = X_test.reshape(len(X_test), N_grid**2)

In [ ]:
cat_y_test = get_y_rank(y_test, categorical=False)

In [ ]:
cat_y_test.shape

In [ ]:
pred = model.predict(X_test.reshape(len(X_test), N_grid**2))

In [ ]:
pred.shape

In [ ]:
prediction = np.argmax(pred, axis=2)

In [ ]:
i=18
for predicted, real in zip (prediction[i] ,cat_y_test[i]):
    print(predicted, real)

In [ ]:
np.argmax(pred[i], axis=1)

In [ ]:
pred = model.predict(X[i].reshape(1,10000))

In [ ]:
pred[0].shape

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
matrix_form = new_X_train.reshape(len(new_X_train),100,100)

In [ ]:
num_classes = 6
# input image dimensions
img_rows, img_cols = 100, 100
input_shape = (img_rows, img_cols,1)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(new_X_train.reshape(len(new_X_train),100,100,1), binary_y, batch_size=128, epochs=10, verbose=2, validation_split = 0.3)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model = Sequential()
model.add(Dense(16, input_shape=(10000,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(8))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

#model.add(Dense(124))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(6))
model.add(Activation('softmax'))

In [ ]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(
          new_X_train.reshape(len(new_X_train),10000),
          binary_y,
          batch_size=128,
          epochs=40,
          verbose=2,
          validation_split = 0.3)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2, n_estimators=50, random_state=0,verbose=1)

In [ ]:
clf.fit(new_X_train.reshape(len(new_X_train),10000), cat_y_train[:,-1])

In [ ]:
clf.score(new_X_train.reshape(len(new_X_train),10000), cat_y_train[:,-1])

In [ ]:
RandomForestClassifier(...)
print(clf.predict([[0, 0, 0, 0]]))